In [1]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import h5py
import scipy

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_dataset():
    train_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/train_catvnoncat.hdf5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/test_catvnoncat.hdf5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes=load_dataset()

In [4]:
#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x_orig.shape))
print("train set y dim= "+str(train_set_y_orig.shape))
print("test set x dim= "+str(test_set_x_orig.shape))
print("test set y dim= "+str(test_set_y_orig.shape))

train set x dim= (209, 64, 64, 3)
train set y dim= (209,)
test set x dim= (50, 64, 64, 3)
test set y dim= (50,)


In [5]:
#reshape dataset
#train data
train_set_x = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
train_set_y = train_set_y_orig.reshape(1,train_set_y_orig.shape[0])
    
#test data
test_set_x = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T
test_set_y = test_set_y_orig.reshape(1,test_set_y_orig.shape[0])
    

In [6]:

#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x.shape))
print("train set y dim= "+str(train_set_y.shape))
print("test set x dim= "+str(test_set_x.shape))
print("test set y dim= "+str(test_set_y.shape))


train set x dim= (12288, 209)
train set y dim= (1, 209)
test set x dim= (12288, 50)
test set y dim= (1, 50)


In [7]:
#normalize data
train_set_x=train_set_x/255
test_set_x=test_set_x/255

In [8]:
def sigmoid(input):
    
    s = 1/(1 + np.exp(-input))
    
    return s

In [9]:
def initialzation():
    
    l1_w = np.random.randn(num_hidden_nodes[0],train_set_x.shape[0])
    l1_b= np.zeros((num_hidden_nodes[0],1))

    l2_w = np.random.randn(num_hidden_nodes[1],num_hidden_nodes[0])
    l2_b= np.zeros((num_hidden_nodes[1],1))
       
    l3_w = np.random.randn(num_hidden_nodes[2],num_hidden_nodes[1])
    l3_b= np.zeros((num_hidden_nodes[2],1))
    
    l4_w = np.random.randn(num_hidden_nodes[3],num_hidden_nodes[2])
    l4_b= np.zeros((num_hidden_nodes[3],1))
    
    lOut_w = np.random.randn(train_set_y.shape[0],num_hidden_nodes[3])
    lOut_b = np.zeros((train_set_y.shape[0],1))
    weights= np.array([l1_w,l2_w,l3_w,l4_w,lOut_w])
    biases= np.array([l1_b,l2_b,l3_b,l4_b,lOut_b])
    
    print(l1_w.shape)
    print(l1_b.shape)
    print(l2_w.shape)
    print(l2_b.shape)
    print(l3_w.shape)
    print(l3_b.shape)
    print(l4_w.shape)
    print(l4_b.shape)
    print(lOut_w.shape)
    print(lOut_b.shape)
    print(weights.shape)
    print(biases.shape)
    return weights,biases

In [24]:
def forwardPropagation(X,weights,biases):
    
    
    z1 = np.dot(weights[0],X) + biases[0]
    A1 = np.tanh(z1)
    z2= np.dot( weights[1],A1) + biases[1]
    A2 = np.tanh(z2)
    z3= np.dot( weights[2],A2) + biases[2]
    A3 = np.tanh(z3)
    z4= np.dot( weights[3],A3) + biases[3]
    A4 = np.tanh(z4)
    
    Z= np.array([z1,z2,z3,z4])
    A= np.array([A1,A2,A3,A4])

    output= np.dot(weights[4],A4) + biases[4]
    output= sigmoid(output)
    cache ={"A":A,
            "Z":Z,
            "output": output} 
    
    return cache
    

In [11]:
def compute_cost (output,X,Y):
    m  =  X.shape[1]
    logs= np.multiply(np.log(output) ,Y) + np.multiply((1-Y),np.log(1-output))
    cost= 1/m * np.sum(logs)
    cost=np.squeeze(cost)
    
    return cost

def backwardPropagation(X,Y,cache):

        
    m  =  X.shape[1]
    A = cache["A"]
    Z = cache["Z"]
    output = cache["output"]

    # BACKWARD PROPAGATION (TO FIND GRAD)
    dzout = output - Y  
    DW = 1/m * np.dot(dzout, A[A.shape[0]-1].T)
    DB = 1/m * np.sum(dzout , axis=1 , keepdims=True)
    
    DW= np.array([DW])
    DB= np.array([DB])   
    print(DW.shape)
    print(DB.shape)
    dz4 = np.multiply(np.dot(weights[weights.shape[0]-1].T,dzout) , (1-np.power(A[A.shape[0]-1],2)))
    dw4 = 1/m * np.dot(dz4,A[A.shape[0]-2].T)
    db4 = 1/m * np.sum(dz4, axis=1 , keepdims=True)
    print(dw4.shape)
    print(db4.shape)
    DW = np.append(DW, [dw4], axis=0)
    DW = np.append(DB, [db4], axis=0)

    zz=dz4
    for i in range(2,A.shape[0]):
        print(i)
        dzi = np.multiply(np.dot(weights[weights.shape[0]-i].T,zz) , (1-np.power(A[A.shape[0]-i],2)))
        dwi = 1/m * np.dot(dzi,A[A.shape[0]-i].T)
        dbi = 1/m * np.sum(dzi, axis=1 , keepdims=True)
        DW = np.append(DW, [dwi], axis=0)
        DW = np.append(DB, [dbi], axis=0)
        zz=dzi
        
    dz1 = np.multiply(np.dot(weights[weights.shape[0]-4].T,zz) , (1-np.power(A[A.shape[0]-4],2)))
    dw1 = 1/m * np.dot(dz1,X.T)
    db1 = 1/m * np.sum(dz1, axis=1 , keepdims=True)
    DW = np.append(DW, [dw1], axis=0)
    DW = np.append(DB, [db1], axis=0)    
    
    print(DW.shape)
    print(DB.shape)
    grads = {"dW": DW,
             "dB": DB}
    
    return grads

In [28]:
def backwardPropagation(X,Y,cache,weights,biases):

        
    m  =  X.shape[1]
    A = cache["A"]
    Z = cache["Z"]
    output = cache["output"]

    # BACKWARD PROPAGATION (TO FIND GRAD)
    dzout = output - Y  
    dw_out = 1/m * np.dot(dzout, A[A.shape[0]-1].T)
    db_out = 1/m * np.sum(dzout , axis=1 , keepdims=True)

    dz4 = np.multiply(np.dot(weights[weights.shape[0]-1].T,dzout) , (1-np.power(A[A.shape[0]-1],2)))
    dw4 = 1/m * np.dot(dz4,A[A.shape[0]-2].T)
    db4 = 1/m * np.sum(dz4, axis=1 , keepdims=True)
    
    dz3 = np.multiply(np.dot(weights[weights.shape[0]-2].T,dz4) , (1-np.power(A[A.shape[0]-2],2)))
    dw3 = 1/m * np.dot(dz3,A[A.shape[0]-3].T)
    db3 = 1/m * np.sum(dz3, axis=1 , keepdims=True)
    
    dz2 = np.multiply(np.dot(weights[weights.shape[0]-3].T,dz3) , (1-np.power(A[A.shape[0]-3],2)))
    dw2 = 1/m * np.dot(dz2,A[A.shape[0]-4].T)
    db2 = 1/m * np.sum(dz2, axis=1 , keepdims=True)
    
    dz1 = np.multiply(np.dot(weights[weights.shape[0]-4].T,dz2) , (1-np.power(A[A.shape[0]-4],2)))
    dw1 = 1/m * np.dot(dz1,X.T)
    db1 = 1/m * np.sum(dz1, axis=1 , keepdims=True)
    grads = {"dw1": dw1,
             "db1": db1,
            "dw2": dw2,
            "db2": db2,
            "dw3": dw3,
            "db3": db3,
            "dw4": dw4,
            "db4": db4,
            "dw_out": dw_out,
            "db_out": db_out}
    
    return grads

In [30]:
def update_parameters(weights,biases,grads,learning_rate=1.2):
    
    
    dw1 = grads["dw1"]
    db1 = grads["db1"]
    dw2 = grads["dw2"]
    db2 = grads["db2"]
    dw3 = grads["dw3"]
    db3 = grads["db3"]
    dw4 = grads["dw4"]
    db4 = grads["db4"]
    dw_out = grads["dw_out"]
    db_out = grads["db_out"]
    weights[0] = weights[0] - learning_rate * dw1
    biases[0] = biases[0] - learning_rate * db1
    weights[1] = weights[1] - learning_rate * dw2
    biases[1] = biases[1] - learning_rate * db2
    weights[2] = weights[2] - learning_rate * dw3
    biases[2] = biases[2] - learning_rate * db3
    weights[3] = weights[3] - learning_rate * dw4
    biases[3] = biases[3] - learning_rate * db4
    weights[4] = weights[4] - learning_rate * dw_out
    biases[4] = biases[4] - learning_rate * db_out       

In [35]:
def predict(weights,biases, X):

    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    
    
    ### make prediction
    z1 = np.dot(weights[0],X) + biases[0]
    A1 = np.tanh(z1)
    z2= np.dot( weights[1],A1) + biases[1]
    A2 = np.tanh(z2)
    z3= np.dot( weights[2],A2) + biases[2]
    A3 = np.tanh(z3)
    z4= np.dot( weights[3],A3) + biases[3]
    A4 = np.tanh(z4)
    
    Z= np.array([z1,z2,z3,z4])
    A= np.array([A1,A2,A3,A4])

    output= np.dot(weights[4],A4) + biases[4]
    output= sigmoid(output)
    ### END CODE HERE ###

    for i in range(output.shape[1]):        
        if output[0,i] >= 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
    
    
    return Y_prediction

In [36]:

n_x=train_set_x.shape[0]
num_hidden_nodes= np.array([80,40,20,10])
n_y=train_set_x.shape[0]
np.random.seed(1)
print(num_hidden_nodes.shape[0])

4


In [37]:
def model(X_train, Y_train, X_test, Y_test,n_h,num_iterations=10000,learning_rate=1.2,print_cost = True):
    
    costs=[]
    weights,biases= initialzation()
    for i in range(num_iterations):
        
        cache = forwardPropagation(X_train,weights,biases)
        
        output = cache["output"]
        cost  = compute_cost (output,X_train,Y_train) 
        grads = backwardPropagation(X_train,Y_train,cache,weights,biases)
        
        update_parameters(weights,biases,grads,learning_rate)
        
                # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    # Predict test/train set examples 
    Y_prediction_test = predict(weights,biases, X_test)
    Y_prediction_train = predict(weights,biases,X_train)

    ### END CODE HERE ###

    # Print train/test accuracy
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "grads" : grads,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [38]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y,4, num_iterations = 200, learning_rate = 0.5)

(80, 12288)
(80, 1)
(40, 80)
(40, 1)
(20, 40)
(20, 1)
(10, 20)
(10, 1)
(1, 10)
(1, 1)
(5,)
(5,)
Cost after iteration 0: -2.126781
Cost after iteration 100: -0.500367
train accuracy: 81.33971291866028 %
test accuracy: 36.0 %
